In [32]:
import os
import pandas as pd
import sqlalchemy
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

connection_string = os.getenv("LINODE_CONN_STRING")
connection_string = os.getenv("LINODE_CONN_STRING")
conn = sqlalchemy.create_engine(os.getenv("LINODE_CONN_STRING")).connect()

In [33]:
season_aggregates = pd.read_sql("SELECT * FROM sports_data.season_averages_by_team", conn)

In [34]:
tourney_games = pd.read_sql("SELECT * FROM sports_data.game_stats_v2", conn)

In [35]:
tourney_games = tourney_games[tourney_games['game_type'] == 'Post-Season Tourney']

In [36]:
tourney_games[tourney_games['season'] == 2023] #spot check that these are the right games

,date,game_type,team,conference,opponent,location,result,adj_offense,adj_defense,offensive_efficiency,...,opponent_offensive_efficiency,opponent_effective_fg_percent,opponent_turnover_percent,opponent_offensive_rebound_percent,opponent_free_throw_rate,opponent_conference,season,Win_Loss,score,opponent_score
261,3/15/23,Post-Season Tourney,Alcorn St.,SWAC,North Texas,A,"L, 69-53",99.1,103.9,86.6,...,112.8,52.0,19.6,34.4,46.0,CUSA,2023,L,53,69
262,3/15/23,Post-Season Tourney,North Texas,CUSA,Alcorn St.,H,"W, 69-53",106.0,92.8,112.8,...,86.6,35.5,18.0,39.0,40.0,SWAC,2023,W,69,53
372,3/17/23,Post-Season Tourney,Arizona St.,P12,TCU,N,"L, 72-70",117.1,99.3,105.0,...,108.0,41.9,7.5,33.3,43.5,B12,2023,L,70,72
373,3/17/23,Post-Season Tourney,TCU,B12,Arizona St.,N,"W, 72-70",116.8,102.2,108.0,...,105.0,55.8,19.5,20.0,34.6,P12,2023,W,72,70
491,3/23/23,Post-Season Tourney,Arkansas,SEC,Connecticut,N,"L, 88-65",105.9,108.1,93.0,...,125.9,65.7,24.3,42.3,38.9,BE,2023,L,65,88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11265,3/21/23,Post-Season Tourney,Wisconsin,B10,Oregon,A,"W, 61-58",101.2,83.3,94.7,...,90.0,49.1,14.0,16.7,21.8,P12,2023,W,61,58
11370,3/24/23,Post-Season Tourney,Texas,B12,Xavier,N,"W, 83-71",124.4,84.0,114.7,...,98.1,49.2,16.6,27.8,18.8,BE,2023,W,83,71
11371,3/24/23,Post-Season Tourney,Xavier,BE,Texas,N,"L, 83-71",108.0,105.5,98.1,...,114.7,58.2,12.4,18.8,31.1,B12,2023,L,71,83
11404,3/14/23,Post-Season Tourney,Vanderbilt,SEC,Yale,H,"W, 71-62",109.2,83.7,101.0,...,88.2,41.7,15.7,28.9,18.2,Ivy,2023,W,71,62


In [37]:
# just need outcomes for these games
tourney_games = tourney_games[['date', 'team', 'opponent', 'season', 'location', 'Win_Loss', 'score', 'opponent_score']]

In [38]:
season_aggregates_team = season_aggregates.copy().add_suffix('_team', axis=1)
tourney_games = pd.merge(tourney_games, season_aggregates_team, left_on=['team', 'season'], right_on=['team_team', 'season_team'], how='left')
tourney_games.drop(columns=['team_team', 'season_team'], inplace=True)

In [39]:
season_aggregates_opponent = season_aggregates.copy().add_suffix('_opponent', axis=1)
tourney_games = pd.merge(tourney_games, season_aggregates_opponent, left_on=['opponent', 'season'], right_on=['team_opponent', 'season_opponent'], how='left')
tourney_games.drop(columns=['team_opponent', 'season_opponent'], inplace=True)

In [41]:
tourney_games = tourney_games[tourney_games['location'] == 'N']

In [42]:
tourney_games.to_sql('tourney_games_with_season_stats', sqlalchemy.create_engine(os.getenv("LINODE_CONN_STRING")), if_exists='replace', schema='sports_data', index=False)

242